In [1]:
import imageio
import numpy as np
import json
from matplotlib import pyplot as plt
from skimage.transform import warp
from scipy.ndimage import gaussian_filter
from scipy import ndimage
import math
import nibabel as nib
import utils
import random
import os

In [2]:
base_dir = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/SIMPLED'

width = 224
height = 224

n_x = 5
n_y = 5

sample_size = 240

p_dropout = 0.7
p_sinus = 0.7
p_rot = 0.7
p_scale = 0.7
p_shear = 0.7
p_global_trans = 0.7
p_local_trans = 0.7

# -----------
dropout_max = 0.2

x_sin_max = 0.1 * height
y_sin_max = 0.05 * width

rot_max = math.pi * 0.1125 # 22.5 deg

x_scale_max = 1.5
y_scale_max = 1.5

x_shear_max = math.pi * 0.05625 # 11.25 deg
y_shear_max = math.pi * 0.05625 # 11.25 deg

x_global_max = 0.3 * width
y_global_max = 0.3 * height

x_local_max = 0.01 * width
y_local_max = 0.01 * height

In [3]:
spacing_x = (width / 2.0) / n_x
spacing_y = (height / 2.0) / n_y

base_offset_x = width / 4.0
base_offset_y = height / 4.0

offset_x = spacing_x / 2.0 + base_offset_x
offset_y = spacing_y / 2.0 + base_offset_y

In [4]:
def random_decision(probability):
    return random.random() < probability

In [5]:
def apply_sinus(x, y, rand_1, rand_2): 
    y_ratio = (y - offset_y) / (spacing_y * (n_y - 1))
    y_sin = math.sin(y_ratio * math.pi) * y_sin_max * rand_1
    u = x - round(y_sin)
    
    x_ratio = (x - offset_x) / (spacing_x * (n_x - 1))
    x_sin = math.sin(x_ratio * math.pi) * x_sin_max * rand_2
    v = y - round(x_sin)
    
    return u,v

def apply_rot(x, y, rand_1):
    shift_x = round(width / 2)
    shift_y = round(height / 2)
    
    x = x - shift_x
    y = y - shift_y
    
    alpha = rot_max * rand_1
    
    vec = np.array([x, y])
    mat = np.array([[np.cos(alpha), -np.sin(alpha)],
                    [np.sin(alpha), np.cos(alpha)]])

    vec = np.dot(mat, vec)
    
    u = int(round(vec[0]))
    v = int(round(vec[1]))
    
    u = u + shift_x
    v = v + shift_y
    
    return u,v

def apply_scale(x, y, rand_1, rand_2):
    rand_1 = (rand_1 + 1.0) / 2.0
    rand_2 = (rand_1 + 1.0) / 2.0
    
    shift_x = round(width / 2)
    shift_y = round(height / 2)
    
    x = x - shift_x
    y = y - shift_y
    
    x_scale = (1 / x_scale_max) + (x_scale_max - (1 / x_scale_max)) * rand_1
    y_scale = (1 / y_scale_max) + (y_scale_max - (1 / y_scale_max)) * rand_2
    
    vec = np.array([x, y])
    mat = np.array([[x_scale, 0      ],
                    [0      , y_scale]])

    vec = np.dot(mat, vec)
    
    u = int(round(vec[0]))
    v = int(round(vec[1]))
    
    u = u + shift_x
    v = v + shift_y
    
    return u,v

def apply_shear(x, y, rand_1, rand_2):
    shift_x = round(width / 2)
    shift_y = round(height / 2)
    
    x = x - shift_x
    y = y - shift_y
    
    phi = x_shear_max * rand_1
    psi = y_shear_max * rand_2
    
    vec = np.array([x, y])
    mat = np.array([[1          , np.tan(phi)],
                    [np.tan(psi), 1          ]])

    vec = np.dot(mat, vec)
    
    u = int(round(vec[0]))
    v = int(round(vec[1]))
    
    u = u + shift_x
    v = v + shift_y
    
    return u,v

def apply_global_trans(x, y, rand_1, rand_2):
    u = x + round(x_global_max * rand_1)
    v = y + round(y_global_max * rand_2)
    
    return u,v

def apply_local_trans(x, y):
    u = x + round(x_local_max * (random.random() * 2.0 - 1.0))
    v = y + round(y_local_max * (random.random() * 2.0 - 1.0))
    
    return u,v

def apply_dropout(rand):
    dropout = dropout_max * rand
    if random_decision(dropout):
        return True
        
    else:
        return False

In [6]:
def get_fixed_data():
    is_dropout = random_decision(p_dropout)
    
    fixed = np.zeros((height, width))
    index_2_xy = dict()
    index = 0
    rand = random.random()
    for index_y in reversed(range(n_y)):
        y = int(round(offset_y + (spacing_y * index_y)))

        for index_x in range(n_x):
            x = int(round(offset_x + (spacing_x * index_x)))
            
            drop = apply_dropout(rand) if is_dropout else False
            
            if not drop:
                fixed[y][x] = 1
                index_2_xy[index] = (x, y)
            
            index += 1
            
    return fixed, index_2_xy

In [7]:
def get_moving_data(rand_1, rand_2, sample_index, index_2_xy):
    is_sinus = random_decision(p_sinus)
    is_rot = random_decision(p_rot)
    is_scale = random_decision(p_scale)
    is_shear = random_decision(p_shear)
    is_global_trans = random_decision(p_global_trans)
    is_local_trans = random_decision(p_local_trans)
    
    moving = np.zeros((height, width))
    index_2_uv = dict()
    index = 0
    for index_y in reversed(range(n_y)):
        y = int(round(offset_y + (spacing_y * index_y)))

        for index_x in range(n_x):
            if index in index_2_xy:
                x = int(round(offset_x + (spacing_x * index_x)))

                u = x
                v = y
                
                if is_sinus: u,v = apply_sinus(u, v, rand_1, rand_2)
                if is_rot: u,v = apply_rot(u, v, rand_1) 
                if is_scale: u,v = apply_scale(u, v, rand_1, rand_2) 
                if is_shear: u,v = apply_shear(u, v, rand_1, rand_2) 
                if is_global_trans: u,v = apply_global_trans(u, v, rand_1, rand_2) 
                if is_local_trans: u,v = apply_local_trans(u, v) 
                
                if u < 0 or v < 0 or u > (width-1) or v > (height-1):
                    print('Additional attempt for sample index "{}"'.format(sample_index))
                    return None, None, False
                
                else:
                    moving[v][u] = 1
                    index_2_uv[index] = (u, v)

            index += 1
            
    return moving, index_2_uv, True

In [8]:
for sample_index in range(0, sample_size):
    # ----- fixed -----
    fixed, index_2_xy = get_fixed_data()

    json_file = json.dumps(index_2_xy)
    f = open(base_dir + "/raw/{}_f.json".format(sample_index), "w")
    f.write(json_file)
    f.close()

    fixed = utils.apply_smoothing(fixed)
    plt.imsave(base_dir + "/raw/{}_f.png".format(sample_index), fixed, cmap="gray")

    # ----- moving -------
    is_valid = False
    
    while(not is_valid):
        rand_1 = random.random() * 2.0 - 1.0
        rand_2 = random.random() * 2.0 - 1.0
        
        moving, index_2_uv, is_valid = get_moving_data(rand_1, rand_2, sample_index, index_2_xy)
        
    json_file = json.dumps(index_2_uv)
    f = open(base_dir + "/raw/{}_m.json".format(sample_index), "w")
    f.write(json_file)
    f.close()

    moving = utils.apply_smoothing(moving)
    plt.imsave(base_dir + "/raw/{}_m.png".format(sample_index), moving, cmap="gray")

Additional attempt for sample index "0"
Additional attempt for sample index "7"
Additional attempt for sample index "10"
Additional attempt for sample index "10"
Additional attempt for sample index "10"
Additional attempt for sample index "11"
Additional attempt for sample index "27"
Additional attempt for sample index "28"
Additional attempt for sample index "28"
Additional attempt for sample index "29"
Additional attempt for sample index "29"
Additional attempt for sample index "33"
Additional attempt for sample index "37"
Additional attempt for sample index "39"
Additional attempt for sample index "43"
Additional attempt for sample index "44"
Additional attempt for sample index "44"
Additional attempt for sample index "48"
Additional attempt for sample index "58"
Additional attempt for sample index "59"
Additional attempt for sample index "63"
Additional attempt for sample index "63"
Additional attempt for sample index "63"
Additional attempt for sample index "65"
Additional attempt

In [9]:
shuffled_list = list(range(240))
random.shuffle(shuffled_list)
train = shuffled_list[0:168] # 70 percent training
validation = shuffled_list[168:204] # 15 percent training
test = shuffled_list[204:240] # 15 percent training

In [10]:
for index in train:
    fixed_sou = os.path.join(base_dir, "raw", "{}_f.png".format(index))
    moving_sou = os.path.join(base_dir, "raw", "{}_m.png".format(index))
    fixed_json_sou = os.path.join(base_dir, "raw", "{}_f.json".format(index))
    moving_json_sou = os.path.join(base_dir, "raw", "{}_m.json".format(index))
    
    fixed_des = os.path.join(base_dir, "train", "{}_f.png".format(index))
    moving_des = os.path.join(base_dir, "train", "{}_m.png".format(index))
    fixed_json_des = os.path.join(base_dir, "train", "{}_f.json".format(index))
    moving_json_des = os.path.join(base_dir, "train", "{}_m.json".format(index))
    
    os.rename(fixed_sou, fixed_des)
    os.rename(moving_sou, moving_des)
    os.rename(fixed_json_sou, moving_json_sou)
    os.rename(moving_json_sou, moving_json_des)

In [11]:
for index in validation:
    fixed_sou = os.path.join(base_dir, "raw", "{}_f.png".format(index))
    moving_sou = os.path.join(base_dir, "raw", "{}_m.png".format(index))
    fixed_json_sou = os.path.join(base_dir, "raw", "{}_f.json".format(index))
    moving_json_sou = os.path.join(base_dir, "raw", "{}_m.json".format(index))
    
    fixed_des = os.path.join(base_dir, "validation", "{}_f.png".format(index))
    moving_des = os.path.join(base_dir, "validation", "{}_m.png".format(index))
    fixed_json_des = os.path.join(base_dir, "validation", "{}_f.json".format(index))
    moving_json_des = os.path.join(base_dir, "validation", "{}_m.json".format(index))
    
    os.rename(fixed_sou, fixed_des)
    os.rename(moving_sou, moving_des)
    os.rename(fixed_json_sou, moving_json_sou)
    os.rename(moving_json_sou, moving_json_des)

In [12]:
for index in test:
    fixed_sou = os.path.join(base_dir, "raw", "{}_f.png".format(index))
    moving_sou = os.path.join(base_dir, "raw", "{}_m.png".format(index))
    fixed_json_sou = os.path.join(base_dir, "raw", "{}_f.json".format(index))
    moving_json_sou = os.path.join(base_dir, "raw", "{}_m.json".format(index))
    
    fixed_des = os.path.join(base_dir, "test", "{}_f.png".format(index))
    moving_des = os.path.join(base_dir, "test", "{}_m.png".format(index))
    fixed_json_des = os.path.join(base_dir, "test", "{}_f.json".format(index))
    moving_json_des = os.path.join(base_dir, "test", "{}_m.json".format(index))
    
    os.rename(fixed_sou, fixed_des)
    os.rename(moving_sou, moving_des)
    os.rename(fixed_json_sou, moving_json_sou)
    os.rename(moving_json_sou, moving_json_des)